In [1]:
import numpy as np
import math

In [2]:
l,d_k,d_v = 4, 8, 8
q = np.random.randn(l, d_k)  # l is number of words in a input my name is ajay 4
k = np.random.randn(l, d_k)  # size of each vectors is 8
v = np.random.randn(l, d_k)

In [3]:
print("Q\n", q)
print("K\n", k)
print("V\n", v)

Q
 [[ 0.31340509  0.55679969  0.03193698  0.5822728   0.45087687 -1.0763082
  -0.79020352 -0.43132768]
 [ 0.74774782  1.19683398  0.74409719 -1.5994998  -0.09815244 -1.01712905
   0.60803932 -0.0090187 ]
 [ 0.81938897  0.04747731  0.04071351  0.11471136  1.48537342 -0.30578805
  -0.29083539 -0.05305894]
 [ 0.82826612 -1.06810048  0.52408215  1.87113025 -0.71062143 -1.33563895
  -0.8284647  -1.16136434]]
K
 [[-1.25788425  0.25182244  0.97716912 -2.07695523 -0.34085481 -1.39546788
   0.93037477  0.69155303]
 [-0.62404629 -0.53228218  1.0345739   1.64439884  1.52175721  0.23191807
   0.11660546 -1.99797336]
 [-1.40787588  1.00486721  0.43193026  2.5152917  -0.19986458  0.48731247
  -0.50466816  1.30512379]
 [ 1.0799233   0.43490982  0.37766703 -1.43097862 -1.34308552 -1.04603835
  -2.4470978  -0.62708596]]
V
 [[-0.77629822  0.14781796  1.47112785  1.16439496 -0.645503   -1.38168076
   1.4121527   0.43674903]
 [-0.66935406 -0.61675843  1.61672049 -0.00638761  0.06507838  0.65538523
  -1.92

In [4]:
q[0]

array([ 0.31340509,  0.55679969,  0.03193698,  0.5822728 ,  0.45087687,
       -1.0763082 , -0.79020352, -0.43132768])

# Self Attention  
self attention = softmax(Q.K.T/sqrt(d_k) + M)V

In [5]:
np.matmul(q, k.T)  

array([[-1.11736077,  1.70472502,  0.81789657,  2.48393102],
       [ 5.42230189, -3.26040971, -4.34655847,  3.61140306],
       [-1.60406525,  1.95570169, -1.16813264, -0.17339011],
       [-4.15283911,  4.50335999,  1.08694034,  3.05748197]])

# so here it's first row is corresponds to my vector and this my has some relation with other words in it's corresponding columns
1. so here my is more focused on name here

Why we need sqrt of D_K : need to minimize the variance and stablize this matmul of Q, K.T

In [6]:
q.var(), k.var(), np.matmul(q, k.T).var()

(0.7012312571928682, 1.423723752396705, 8.371570150689042)

In [7]:
scaled = np.matmul(q, k.T)/np.sqrt(d_k)

In [8]:
scaled.var()

1.04644626883613

In [9]:
scaled

array([[-0.39504669,  0.60271131,  0.28917011,  0.87820223],
       [ 1.91707322, -1.15272891, -1.53674048,  1.2768238 ],
       [-0.56712271,  0.69144496, -0.41299726, -0.06130266],
       [-1.46825035,  1.59217819,  0.38429144,  1.08098312]])

# Masking : so that we can ensure the words don't get the context from words generated in future 
1. not required in decorder but in encoder cuz input is going simulatenously not in encoder one by one

In [10]:
mask = np.tril(np.ones((l,l) )) # traingular matrix with one 

In [11]:
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

1. here first row represents my 
2. now second focus on my name 
3. my name is and so on

In [12]:
mask = np.tril(np.ones((l,l) )) # traingular matrix with one 
mask[mask == 0] = -np.infty
mask[mask == 1] = 0

In [13]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [14]:
scaled + mask

array([[-0.39504669,        -inf,        -inf,        -inf],
       [ 1.91707322, -1.15272891,        -inf,        -inf],
       [-0.56712271,  0.69144496, -0.41299726,        -inf],
       [-1.46825035,  1.59217819,  0.38429144,  1.08098312]])

# Softmax

In [15]:
def softmax(x):
    return (np.exp(x).T/ np.sum(np.exp(x), axis = 1)).T

In [16]:
attention = softmax(scaled + mask) #that's why mask is need we didn't need te context of generated words 

In [17]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.95562978, 0.04437022, 0.        , 0.        ],
       [0.17583924, 0.61902016, 0.2051406 , 0.        ],
       [0.02409058, 0.51401353, 0.15360169, 0.3082942 ]])

In [18]:
new_v = np.matmul(attention,v)

In [19]:
new_v

array([[-0.77629822,  0.14781796,  1.47112785,  1.16439496, -0.645503  ,
        -1.38168076,  1.4121527 ,  0.43674903],
       [-0.77155308,  0.11389354,  1.47758783,  1.11244708, -0.61397435,
        -1.2912957 ,  1.26388346,  0.4435556 ],
       [-0.20133397, -0.19387296,  1.30249797,  0.32565515, -0.02912318,
        -0.18042063, -1.04448679,  0.17189906],
       [ 0.00953626,  0.05922697,  1.0075256 , -0.5556328 , -0.39317888,
        -0.32927121, -0.91902759, -0.01195853]])

In [20]:
v

array([[-0.77629822,  0.14781796,  1.47112785,  1.16439496, -0.645503  ,
        -1.38168076,  1.4121527 ,  0.43674903],
       [-0.66935406, -0.61675843,  1.61672049, -0.00638761,  0.06507838,
         0.65538523, -1.92948627,  0.59015314],
       [ 1.70377469,  0.78931591,  0.20977516,  0.60866978,  0.21495862,
        -1.67282148, -0.47970878, -1.31721971],
       [ 0.35872263,  0.81561014,  0.35306294, -2.18587685, -1.4404991 ,
        -1.21933652,  0.36464831, -0.40059006]])

In [21]:
# after using attention we can see my having context and next word getting so much of context 

In [22]:
def softmax(x):
    return (np.exp(x).T/ np.sum(np.exp(x), axis = 1)).T

def scaled_dot_product(q, k, v, mask):
    d_k = q.shape[-1]

    scaled = np.matmul(q, k.T)/ np.sqrt(d_k)
    attention = softmax(scaled + mask)
    
    output_v = np.matmul(attention, v)
    
    return output_v, attention

In [23]:
q = np.random.randn(l, d_k)  # l is number of words in a input my name is ajay 4
k = np.random.randn(l, d_k)  # size of each vectors is 8
v = np.random.randn(l, d_k)
mask = np.tril(np.ones((q.shape[0], q.shape[0]) )) # traingular matrix with one 
mask[mask == 0] = -np.infty
mask[mask == 1] = 0
vector , attention = scaled_dot_product(q, k, v, mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)

Q
 [[-2.82239293 -0.47236277 -0.04448361  1.06968885 -0.35909733 -0.63237679
  -0.96846493 -0.92082198]
 [ 0.7993451  -0.05727779 -0.77572556  0.67483218 -1.19270505  1.49999854
   0.20769235  1.19580674]
 [-0.73461647  0.79470803  0.90457411 -0.13794276  0.2626833  -0.4164732
  -0.80817525  1.73471066]
 [ 1.01259856 -0.64249233  0.19130425  0.8676278   1.26207273  0.0970803
  -0.59029314 -0.11829099]]
K
 [[ 0.73585521 -0.53771955  0.08794401  1.01987147  0.22819695  0.88969509
   0.24045278  0.16817388]
 [-0.53384289 -1.05438514  0.49406938 -0.79753462 -0.66319722  0.74923826
  -0.33447856  0.4277864 ]
 [ 2.49487759  1.23208374 -1.25320365  0.86076107  0.24320857 -0.04221494
  -1.45408512 -0.07658606]
 [-0.38344551  0.1520428   1.35336729 -0.07491407  2.08894245 -1.97275519
   0.73294222 -0.27002603]]
V
 [[ 0.10950061 -0.02226883  1.84966235  0.11710753  1.31396154  1.43469362
   1.03162257 -0.22330707]
 [ 0.40759364  1.68468131 -0.89672188 -0.1986948  -2.03949484  0.45643565
   0.179

In [24]:
vector, attention

(array([[ 1.09500611e-01, -2.22688285e-02,  1.84966235e+00,
          1.17107535e-01,  1.31396154e+00,  1.43469362e+00,
          1.03162257e+00, -2.23307071e-01],
        [ 2.20337026e-01,  6.12406322e-01,  8.28506677e-01,
         -3.13525757e-04,  6.70853863e-02,  1.07095949e+00,
          7.14625975e-01,  1.16734615e-03],
        [ 5.63063973e-01,  6.90895200e-01, -5.78694496e-02,
         -9.57350436e-02, -2.26145922e-01,  8.95279997e-01,
          2.33108550e-01,  3.96608731e-01],
        [ 4.12659503e-01, -1.05584423e-01,  1.13462016e-01,
         -1.07582381e-02,  7.94313960e-01,  6.12791726e-01,
          9.25749082e-03,  2.75732361e-01]]),
 array([[1.        , 0.        , 0.        , 0.        ],
        [0.62818179, 0.37181821, 0.        , 0.        ],
        [0.24753429, 0.48109795, 0.27136776, 0.        ],
        [0.27043676, 0.08472199, 0.41440458, 0.23043667]]))